In [2]:
import numpy as np
import pandas as pd
import river
from river import preprocessing, stats, compose, metrics
from river import drift, ensemble, forest
from river.drift import ADWIN
from river.compose import Pipeline
from river import stats

**Pre-processing pipeline with imputing, standardizing, and encoding**

In [ ]:
def preprocess_data():
  #EXTRACTING
  dataset = pd.read_excel('../Dataset/ttc-bus-delay-data-2023.xlsx')
  X = dataset.iloc[:, :-1] #or whatever column we want the INDEPENDENT variables (Weather, route number, time of day, etc.)
  y = dataset.iloc[:, -1].values #or whatever column we want the DEPENDENT variables (I'm guessing the delay time)


  print(X)

  #IMPUTING
  imputer = preprocessing.StatImputer(('Direction', stats.Mode()))

  # Assuming d_data is extracted from X['Direction']

  # Create a dataframee of only the direction column

  X.replace({np.nan: None}, inplace=True) # Still counts nan as missing. replace in place.


# Create imputer
  imputer = preprocessing.StatImputer(('Direction', stats.Mode()))

  for index, row in X.iterrows():
    # Create a dictionary row
    row_dict = {'Direction': row['Direction']}

    # Only train the imputer on non-None/na values
    if row['Direction'] is not None:
        imputer.learn_one(row_dict)

    transformed_row = imputer.transform_one(row_dict)

    # Alter the original dataset
    X.at[index, 'Direction'] = transformed_row.get('Direction', row['Direction'])
  print(X)
 

  #SCALING
  scaler = preprocessing.StandardScaler()

  for index, row in X.iterrows():
      row_dict = row.to_dict()

      scaler.learn_one({"Min Delay": row["Min Delay"]})

      scaled_value = scaler.transform_one({"Min Delay": row["Min Delay"]})

      X.at[index, "Min Delay"] = scaled_value["Min Delay"]
  print(X)
  

  #ENCODING
  encoder = preprocessing.OneHotEncoder()

  for index, row in X.iterrows():

      row_dict = row.to_dict()

      encoder.learn_one({"Incident": row["Incident"]})

      encoded_row = encoder.transform_one({"Incident": row["Incident"]})

      for key, value in encoded_row.items():
          if key not in X.columns:
              X[key] = 0

          X.at[index, key] = value

  print(X)
  

In [11]:
preprocess_data()

            Date Route   Time     Day               Location  \
0     2023-01-01    91  02:30  Sunday  WOODBINE AND MORTIMER   
1     2023-01-01    69  02:34  Sunday         WARDEN STATION   
2     2023-01-01    35  03:06  Sunday           JANE STATION   
3     2023-01-01   900  03:14  Sunday        KIPLING STATION   
4     2023-01-01    85  03:43  Sunday         MEADOWALE LOOP   
...          ...   ...    ...     ...                    ...   
56202 2023-12-31    41  01:26  Sunday     KEELE AND EGLINTON   
56203 2023-12-31    94  01:31  Sunday   CASTLE FRANK STATION   
56204 2023-12-31    63  01:40  Sunday    OAKWOOD AND VAUGHAN   
56205 2023-12-31    34  01:54  Sunday       EGLINTON STATION   
56206 2023-12-31    41  01:55  Sunday       KEELE AND ROGERS   

                    Incident  Min Delay  Min Gap Direction  
0                  Diversion         81      111       NaN  
1                   Security         22       44         S  
2      Cleaning - Unsanitary         30       60

/var/folders/cp/gq04t2hd05q_ms7bnky_j7940000gn/T/ipykernel_4527/818312498.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5484943744017863' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.at[index, "Min Delay"] = scaled_value["Min Delay"]


            Date Route   Time     Day               Location  \
0     2023-01-01    91  02:30  Sunday  WOODBINE AND MORTIMER   
1     2023-01-01    69  02:34  Sunday         WARDEN STATION   
2     2023-01-01    35  03:06  Sunday           JANE STATION   
3     2023-01-01   900  03:14  Sunday        KIPLING STATION   
4     2023-01-01    85  03:43  Sunday         MEADOWALE LOOP   
...          ...   ...    ...     ...                    ...   
56202 2023-12-31    41  01:26  Sunday     KEELE AND EGLINTON   
56203 2023-12-31    94  01:31  Sunday   CASTLE FRANK STATION   
56204 2023-12-31    63  01:40  Sunday    OAKWOOD AND VAUGHAN   
56205 2023-12-31    34  01:54  Sunday       EGLINTON STATION   
56206 2023-12-31    41  01:55  Sunday       KEELE AND ROGERS   

                    Incident  Min Delay  Min Gap Direction  
0                  Diversion   0.000000      111      None  
1                   Security  -1.000000       44         S  
2      Cleaning - Unsanitary  -0.548494       60